In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Products
    limit 2
    ''',
    con,
)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18
1,2,Chang,1,1,24 - 12 oz bottles,19


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

### 1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [7]:
# Апдейт от 30.01 - убрал из 3 строки "..Country in Madrid.."

In [8]:
tasks+=1

pd.read_sql(
    '''
    SELECT CustomerName, Country, Address, City
    FROM Customers
    WHERE Country in ("Germany","France") OR city = "Madrid" --Апдейт
    Limit 10
    ''',
    con,
)

,CustomerName,Country,Address,City
0,Alfreds Futterkiste,Germany,Obere Str. 57,Berlin
1,Blauer See Delikatessen,Germany,Forsterstr. 57,Mannheim
2,Blondel père et fils,France,"24, place Kléber",Strasbourg
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67",Madrid
4,Bon app',France,"12, rue des Bouchers",Marseille
5,Drachenblut Delikatessend,Germany,Walserweg 21,Aachen
6,Du monde entier,France,"67, rue des Cinquante Otages",Nantes
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86",Madrid
8,Folies gourmandes,France,"184, chaussée de Tournai",Lille
9,Frankenversand,Germany,Berliner Platz 43,München


### 2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, Count(Country) as Count 
    FROM Customers 
    GROUP BY Country
    ORDER BY Count(*) DESC
    limit 3
    ''',
    con,
)

,Country,Count
0,USA,13
1,France,11
2,Germany,11


### 3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [10]:
tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate
    FROM Shippers 
    left join Orders
    on Orders.ShipperID = Shippers.ShipperID
    ORDER BY OrderID ASC
    LIMIT 1 OFFSET 9
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


### 4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [11]:
# Апдейт от 30.01 для создания агрегата использовал умножение. 10ая строка запроса

In [12]:
tasks+=1

pd.read_sql(
    '''
    SELECT ProductName, Price
    FROM Products
    join OrderDetails
    on Products.ProductID = OrderDetails.ProductID
    where OrderID = 
        (
            select a.OrderID as OrderID
            from
            (
            SELECT *, sum(Price*Quantity) as 'sum_of_order' -- Апдейт
            FROM Products
            join OrderDetails
            on Products.ProductID = OrderDetails.ProductID
            group by OrderID
            order by sum_of_order desc
            limit 1
            ) as a
        )
    ''',
    con,
)

,ProductName,Price
0,Sir Rodney's Marmalade,81
1,Côte de Blaye,263.5
2,Camembert Pierrot,34
3,Mozzarella di Giovanni,34.8


### 5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT OrderID, ProductName, Quantity
    FROM Products
    join OrderDetails
    on Products.ProductID = OrderDetails.ProductID
    where ProductName in
            (
                select a.ProductName as ProductName
                from
                    (SELECT ProductName,sum(Quantity) as Quantity
                    FROM Products
                    join OrderDetails
                    on Products.ProductID = OrderDetails.ProductID
                    group by ProductName
                    order by Quantity desc
                    limit 1) a
            )
    group by OrderID
    order by Quantity desc
    ''',
    con,
)

,OrderID,ProductName,Quantity
0,10359,Gorgonzola Telino,70
1,10390,Gorgonzola Telino,60
2,10342,Gorgonzola Telino,56
3,10376,Gorgonzola Telino,42
4,10272,Gorgonzola Telino,40
5,10393,Gorgonzola Telino,32
6,10356,Gorgonzola Telino,30
7,10374,Gorgonzola Telino,30
8,10335,Gorgonzola Telino,25
9,10253,Gorgonzola Telino,20


### 6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT SupplierName, Country, ContactName, Phone
    FROM Suppliers
    where SupplierID in 
        (
            select a.SupplierID
            from
            (SELECT SupplierID, count(OrderID) as Sum_Orders
            FROM Products
            JOIN OrderDetails
            on Products.ProductID = OrderDetails.ProductID
            group by SupplierID
            order by Sum_Orders desc
            limit 5) as a
        )
    ''', 
    con, 
)

,SupplierName,Country,ContactName,Phone
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
1,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
2,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
3,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323
4,Norske Meierier,Norway,Beate Vileid,(0)2-953010


### 7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [15]:
# Апдейт от 30.01 самую дорогую категорию искал с помощью умножения. 1ая строка запроса

In [16]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, CategoryName, sum(Price*Quantity) as 'sum_by_orders' --Апдейт

    FROM Products
    join OrderDetails
        on Products.ProductID = OrderDetails.ProductID
    join Categories
        on Products.CategoryID = Categories.CategoryID
    join Orders
        on Orders.OrderID = OrderDetails.OrderID
    join Customers
        on Customers.CustomerID = Orders.CustomerID
    
    where Country = 'Brazil'
    group by Country, CategoryName
    order by sum_by_orders desc
    limit 1
    ''',
    con,
)

,Country,CategoryName,sum_by_orders
0,Brazil,Beverages,13636.0


### 8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [17]:
# Апдейт от 30.01 ищем самый дорогой/дешевый заказ с помощью 'sum(Price*Quantity)'

In [18]:
tasks+=1
pd.read_sql(
    '''
    
        SELECT *
        FROM 
        (select Country, c.OrderID as OrderID, sum(Price*Quantity) as 'sum_order_price' --Апдейт
        from Customers a
        join Orders b
            on a.CustomerID = b.CustomerID 
        join OrderDetails c 
            on b.OrderID = c.OrderID 
        join Products d
            on c.ProductID = d.ProductID 
            
            where Country = 'USA'
            group by c.OrderID
            order by sum_order_price desc
            limit 1) as e
    
    union all
    
        SELECT *
        FROM 
        (select Country, c.OrderID as OrderID, sum(Price*Quantity) as 'sum_order_price' --Апдейт
        from Customers a
        join Orders b
            on a.CustomerID = b.CustomerID 
        join OrderDetails c 
            on b.OrderID = c.OrderID 
        join Products d
            on c.ProductID = d.ProductID 
            
            where Country = 'USA'
            group by c.OrderID
            order by sum_order_price asc
            limit 1) as e
    
    ''',
    con,
)

,Country,OrderID,sum_order_price
0,USA,10324,7698.45
1,USA,10349,0.00


### 9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [19]:
# В этом задании я знатно считерил :) так как уже много что в интернете перерыл и идей больше не было, но считаю что так удалось 
# Согласно ТЗ дать частично хороший результат)

with con:
    cur = con.cursor()   
    cur.execute("UPDATE Employees SET Photo = '1968.12' WHERE Photo = 'EmpID1.pic'")
    cur.execute("UPDATE Employees SET Photo = '1952.02' WHERE Photo = 'EmpID2.pic'")
    
    cur.execute("UPDATE Employees SET Photo = '1963.08' WHERE Photo = 'EmpID3.pic'")
    cur.execute("UPDATE Employees SET Photo = '1958.09' WHERE Photo = 'EmpID4.pic'")
    cur.execute("UPDATE Employees SET Photo = '1955.03' WHERE Photo = 'EmpID5.pic'")
    cur.execute("UPDATE Employees SET Photo = '1963.07' WHERE Photo = 'EmpID6.pic'")
    cur.execute("UPDATE Employees SET Photo = '1960.05' WHERE Photo = 'EmpID7.pic'")
    cur.execute("UPDATE Employees SET Photo = '1958.01' WHERE Photo = 'EmpID8.pic'")
    cur.execute("UPDATE Employees SET Photo = '1969.07' WHERE Photo = 'EmpID9.pic'")
    cur.execute("UPDATE Employees SET Photo = '1928.09' WHERE Photo = 'EmpID10.pic'")

In [20]:
# Апдейт от 30.01 - почему 4 сотрудника вывел, потому что мой чит ход не дал до конца решить задачу. У меня самые 
# молодые сотрудники в выборке получились в строке 1, 2, 4. На 3 месте сотрудник, у которого ДР в августе, а на 4 у которого 
# в июле

In [21]:
tasks+=1
pd.read_sql(
    '''
    select sum_orders, (FirstName || " " ||  LastName) as 'Full_Name', BirthDate
    from
    (SELECT LastName, FirstName, Photo as Birth_info , BirthDate, count(OrderID) as 'sum_orders'
    FROM Employees a
    join Orders b
    on a.EmployeeID = b.EmployeeID
    group by FirstName
    order by Photo desc
    limit 4 -- Апдейт
    ) as a
    ''',
    con,
)

,sum_orders,Full_Name,BirthDate
0,6,Anne Dodsworth,02.07.1969
1,29,Nancy Davolio,08.12.1968
2,31,Janet Leverling,30.08.1963
3,18,Michael Suyama,02.07.1963


In [22]:
# Почему не использовал "order by BirthDate asc", потому что сколько бы не гуглил, не нашел способа фильтрации даты
# В текущем массиве он фильтрует по первой паре чисел дня рождения (месяц и год почему то не участвуют)

In [23]:
pd.read_sql(
    '''
    select sum_orders, (FirstName || " " ||  LastName) as 'Full_Name', BirthDate
    from
    (SELECT LastName, FirstName, Photo as Birth_info , BirthDate, count(OrderID) as 'sum_orders'
    FROM Employees a
    join Orders b
    on a.EmployeeID = b.EmployeeID
    group by FirstName
    order by BirthDate asc
    limit 10
    ) as a
    ''',
    con,
)

,sum_orders,Full_Name,BirthDate
0,18,Michael Suyama,02.07.1963
1,6,Anne Dodsworth,02.07.1969
2,11,Steven Buchanan,04.03.1955
3,29,Nancy Davolio,08.12.1968
4,27,Laura Callahan,09.01.1958
5,20,Andrew Fuller,19.02.1952
6,40,Margaret Peacock,19.09.1958
7,14,Robert King,29.05.1960
8,31,Janet Leverling,30.08.1963


### 10. Сколько банок крабового мяса всего было заказано.

In [24]:
# Апдейт от 30.01 - внес изменения в 1ую строку запроса

In [25]:
tasks+=1
pd.read_sql(
    '''
    SELECT ProductName, sum(Quantity*Unit) as 'Quantity_1' -- Апдейт
    FROM OrderDetails a
    join Products b
    on a.ProductID = b.ProductID
    where ProductName like '%Crab%'
    ''',
    con,
)

,ProductName,Quantity_1
0,Boston Crab Meat,6144


In [26]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
